In [1]:
import os
from transformers import AutoTokenizer, AutoModel
import torch
import pinecone

In [2]:
model_name = "BAAI/bge-base-en-v1.5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

d:\pytorch_gpu_env\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--BAAI--bge-base-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [3]:
def generate_embedding(text):
    """Generate embeddings for a given text."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings.squeeze().numpy()

In [4]:
def load_data(base_folder):
    """Load judgments and URLs from text files in the base folder."""
    data = []
    for folder_name in os.listdir(base_folder):
        folder_path = os.path.join(base_folder, folder_name)
        if os.path.isdir(folder_path):
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)
                if file_name.endswith(".txt"):
                    with open(file_path, "r", encoding="utf-8") as file:
                        url = file.readline().strip()
                        content = file.read().strip()
                        data.append({"url": url, "text": content})
    return data

In [14]:
# Pinecone(api_key="pcsk_4b3hm7_NbZGNPRPDSvAoJECCMG93FLyaQM9ZRBTbT8hToBFNsLsJ8baK7gpvEpPwF3P2EA")
from pinecone import Pinecone, ServerlessSpec
pc=Pinecone(api_key="pcsk_4b3hm7_NbZGNPRPDSvAoJECCMG93FLyaQM9ZRBTbT8hToBFNsLsJ8baK7gpvEpPwF3P2EA")
index_name = "judgment-search"
if index_name not in pc.list_indexes():
    pc.create_index(index_name, dimension=768 ,spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1")
        )  # Adjust dimension based on model
index = pc.Index(index_name)

In [15]:
def store_in_pinecone(data):
    """Generate embeddings and store data in Pinecone."""
    for i, item in enumerate(data):
        embedding = generate_embedding(item["text"])
        index.upsert([(f"doc-{i}", embedding, {"url": item["url"], "text": item["text"][:200]})])


In [26]:
def query_pinecone(query_text, top_k=10):
    """Query Pinecone for similar texts."""
    # Generate the embedding for the query text
    query_embedding = generate_embedding(query_text)
    
    # Perform the query using keyword arguments
    results = index.query(vector=query_embedding.tolist(), top_k=top_k, include_metadata=True)
    
    return results


In [ ]:
base_folder = "writ"
# Load data
data = load_data(base_folder)
print(f"Loaded {len(data)} judgments.")

    # Store data in Pinecone
store_in_pinecone(data)
print("Data stored in Pinecone.")

#     # Example query
# query_text = "Sample legal judgment text to search for similar cases."
#     results = query_pinecone(query_text)
#     for result in results["matches"]:
#         print(f"Score: {result['score']}, URL: {result['metadata']['url']}")

Loaded 1922 judgments.
Data stored in Pinecone.


ValueError: The argument order for `query()` has changed; please use keyword arguments instead of positional arguments. Example: index.query(vector=[0.1, 0.2, 0.3], top_k=10, namespace='my_namespace')

In [29]:
query_text = "A case involving the violation of fundamental rights under Article 21 of the Constitution, where the petitioner seeks the issuance of a writ of habeas corpus for the unlawful detention of an individual."
results = query_pinecone(query_text,top_k=10)
for result in results["matches"]:
    print(f"Score: {result['score']}, URL: {result['metadata']['url']}")

Score: 0.828869224, URL: https://indiankanoon.org/docfragment/953550/?formInput=habeas%20corpus%20%20doctypes%3A%20judgments
Score: 0.822812796, URL: https://indiankanoon.org/docfragment/65444816/?formInput=habeas%20corpus%20%20doctypes%3A%20judgments
Score: 0.820750475, URL: https://indiankanoon.org/docfragment/161055323/?formInput=habeas%20corpus%20%20doctypes%3A%20judgments
Score: 0.817860186, URL: https://indiankanoon.org/docfragment/89846666/?formInput=habeas%20corpus%20%20doctypes%3A%20judgments
Score: 0.816762626, URL: https://indiankanoon.org/docfragment/107270237/?formInput=habeas%20corpus%20%20doctypes%3A%20judgments
Score: 0.813470066, URL: https://indiankanoon.org/docfragment/884325/?formInput=habeas%20corpus%20%20doctypes%3A%20judgments
Score: 0.812360823, URL: https://indiankanoon.org/docfragment/19974910/?formInput=habeas%20corpus%20%20doctypes%3A%20judgments
Score: 0.810760796, URL: https://indiankanoon.org/docfragment/14171249/?formInput=habeas%20corpus%20%20doctypes%3